### Merge: change the folder names correspond to `object name`

In [86]:
import os

parent_folder = './asset/ycb_rename'  

for root, dirs, files in os.walk(parent_folder):
    for folder in dirs:
        folder_path = os.path.join(root, folder)

        for filename in os.listdir(folder_path):
            if filename.startswith('textured'):
                file_path = os.path.join(folder_path, filename)  # full path of the file

                parent_folder_name = file_path.split('/')[-3].split('_')[1:]
                parent_folder_name = '_'.join(parent_folder_name)
                print(f"folder: {folder}")
                new_filename = filename.replace('textured', parent_folder_name)
                new_file_path = os.path.join(folder_path, new_filename)  # new full path of the file
                os.rename(file_path, new_file_path)  # Rename the file
                print(f"file_path: {file_path}")
                print(f"new_file_path: {new_file_path}")

            if filename.startswith('texture_map'):
                file_path = os.path.join(folder_path, filename)  # full path of the file

                parent_folder_name = file_path.split('/')[-3].split('_')[1:]
                parent_folder_name = '_'.join(parent_folder_name)
                print(f"folder: {folder}")
                new_filename = filename.replace('texture_map', parent_folder_name)
                new_file_path = os.path.join(folder_path, new_filename)  # new full path of the file
                os.rename(file_path, new_file_path)  # Rename the file
                print(f"file_path: {file_path}")
                print(f"new_file_path: {new_file_path}")

folder: poisson
file_path: ./ycb_test/061_foam_brick/poisson/textured.png
new_file_path: ./ycb_test/061_foam_brick/poisson/foam_brick.png
folder: poisson
file_path: ./ycb_test/061_foam_brick/poisson/textured.mtl
new_file_path: ./ycb_test/061_foam_brick/poisson/foam_brick.mtl
folder: poisson
file_path: ./ycb_test/061_foam_brick/poisson/textured.obj
new_file_path: ./ycb_test/061_foam_brick/poisson/foam_brick.obj
folder: tsdf
file_path: ./ycb_test/061_foam_brick/tsdf/textured.png
new_file_path: ./ycb_test/061_foam_brick/tsdf/foam_brick.png
folder: tsdf
file_path: ./ycb_test/061_foam_brick/tsdf/textured.mtl
new_file_path: ./ycb_test/061_foam_brick/tsdf/foam_brick.mtl
folder: tsdf
file_path: ./ycb_test/061_foam_brick/tsdf/textured.obj
new_file_path: ./ycb_test/061_foam_brick/tsdf/foam_brick.obj
folder: google_16k
file_path: ./ycb_test/061_foam_brick/google_16k/textured.mtl
new_file_path: ./ycb_test/061_foam_brick/google_16k/foam_brick.mtl
folder: google_16k
file_path: ./ycb_test/061_foam_br

### Merge: `obj, mtl, png` files into one folder

In [4]:
import os
import shutil

# 상위 디렉토리 경로 설정
src_folder = './asset/ycb_rename'
dest_folder = './asset/ycb_merged'

# 목적지 폴더 생성 (이미 존재하는 경우, 그냥 지나감)
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

# 우선순위에 따른 하위 폴더 순회
subfolders_priority = ['google_16k', 'tsdf', 'poisson', 'clouds']

# 상위 디렉토리 내의 모든 object 폴더에 대해
for object_folder in os.listdir(src_folder):
    object_folder_path = os.path.join(src_folder, object_folder)
    if os.path.isdir(object_folder_path):
        # 하위 폴더를 우선순위에 따라 순회
        for subfolder in subfolders_priority:
            subfolder_path = os.path.join(object_folder_path, subfolder)
            if os.path.exists(subfolder_path):
                # 해당 하위 폴더에서 모든 파일을 순회
                for file in os.listdir(subfolder_path):
                    # 원본 파일의 절대 경로
                    src_file_path = os.path.join(subfolder_path, file)
                    # 목적지 파일의 절대 경로
                    dest_file_path = os.path.join(dest_folder, file)
                    # 동일한 이름의 파일이 목적지 폴더에 이미 존재하는 경우 이름을 변경합니다.
                    if os.path.exists(dest_file_path):
                        base, extension = os.path.splitext(file)
                        i = 1
                        while os.path.exists(dest_file_path):
                            file = f"{base}_{i}{extension}"
                            dest_file_path = os.path.join(dest_folder, file)
                            i += 1
                    # 파일 복사
                    shutil.copy2(src_file_path, dest_file_path)
                    # mtl 파일의 경우, 특정 줄을 수정
                    if file.endswith('.mtl'):
                        with open(dest_file_path, 'r') as infile:
                            lines = infile.readlines()
                        with open(dest_file_path, 'w') as outfile:
                            for line in lines:
                                if line.startswith('map_Kd'):
                                    line = f'map_Kd {os.path.splitext(file)[0]}.png\n'
                                outfile.write(line)
                    # obj 파일의 경우, 특정 줄을 수정
                    elif file.endswith('.obj'):
                        with open(dest_file_path, 'r') as infile:
                            lines = infile.readlines()
                        with open(dest_file_path, 'w') as outfile:
                            for line in lines:
                                if line.startswith('mtllib'):
                                    line = f'mtllib {os.path.splitext(file)[0]}.mtl\n'
                                outfile.write(line)
                # 하위 폴더에서 한 번이라도 파일을 복사했다면, 다음 object 폴더로 이동
                break

print("All files are merged and mtl/obj files are modified in the folder: ", dest_folder)


All files are merged and mtl/obj files are modified in the folder:  ycb_merged


### And run the below line in `/ycb_merged` folder
##### `obj2mjcf --obj-dir ./ --save-mjcf --save-mtl --vhacd-args.enable`



### Merge v2: Converted mjcf from obj files [xml, obj, png] into each folder

In [25]:
import os
import shutil

src_folder = 'asset/ycb_merged'     # 원래 파일들이 있는 폴더
dst_folder = 'asset/ycb_merged_v2'  # 이동할 파일들이 들어갈 폴더
extensions = ['obj', 'png', 'xml']  # 각 확장자 이름들

# 새로운 폴더가 없다면 생성합니다.
if not os.path.exists(dst_folder):
    os.mkdir(dst_folder)

# 모든 하위 디렉터리를 순회하면서 파일을 찾습니다.
for dirpath, dirnames, filenames in os.walk(src_folder):
    for filename in filenames:
        # 파일 확장자를 얻습니다.
        file_extension = os.path.splitext(filename)[1][1:]

        # 확장자가 우리가 관심 있는 목록에 있다면 이동합니다.
        if file_extension in extensions:
            # 새로운 폴더 경로를 생성합니다. (예: 'ycb_merged_v2/obj/')
            new_folder_path = os.path.join(dst_folder, file_extension)
            # 새로운 폴더가 없다면 생성합니다.
            if not os.path.exists(new_folder_path):
                os.mkdir(new_folder_path)
            
            # 원본 파일 경로와 새 파일 경로를 생성합니다.
            old_file_path = os.path.join(dirpath, filename)
            new_file_path = os.path.join(new_folder_path, filename)

            # 원본 파일을 새 위치로 이동합니다.
            shutil.move(old_file_path, new_file_path)


### Modifying the lines about `<default>, <material>, <geom>` in xml files

In [26]:
import os
import xml.etree.ElementTree as ET

def modify_xml(xml_filepath):
    # XML 파일 파싱
    tree = ET.parse(xml_filepath)
    root = tree.getroot()

    # 'default' 태그를 찾아서 제거
    for default_tag in root.findall('default'):
        root.remove(default_tag)

    # 'asset' 태그 내의 'material' 태그의 'name' 속성을 'texture' 속성과 동일하게 설정
    for asset_tag in root.findall('asset'):
        for material_tag in asset_tag.findall('material'):
            if 'texture' in material_tag.attrib:
                material_tag.set('name', material_tag.get('texture'))

    # 'worldbody' 태그 내부의 'body' 태그의 'name' 속성을 찾아 'geom' 태그의 'material' 속성을 그 값으로 설정
    for worldbody_tag in root.findall('worldbody'):
        for body_tag in worldbody_tag.findall('body'):
            body_name = body_tag.get('name')
            for geom_tag in body_tag.findall('geom'):
                geom_tag.set('material', body_name)

    # 수정된 트리를 다시 파일로 쓰기
    tree.write(xml_filepath)

merged_folder = 'asset/ycb_merged_v2/'

# 모든 object 폴더에 대해
for object_folder in os.listdir(merged_folder):
    object_folder_path = os.path.join(merged_folder, object_folder)
    if os.path.isdir(object_folder_path):
        # 해당 object 폴더 내의 모든 XML 파일에 대해
        for xml_file in os.listdir(object_folder_path):
            if xml_file.endswith('.xml'):
                print(f"xml_file: {xml_file}")
                modify_xml(os.path.join(object_folder_path, xml_file))


xml_file: kinbody_43.xml
xml_file: windex_bottle.xml
xml_file: toy_airplane_4.xml
xml_file: kinbody_31.xml
xml_file: kinbody_76.xml
xml_file: cups_9.xml
xml_file: kinbody_2.xml
xml_file: lego_duplo_12.xml
xml_file: lego_duplo_7.xml
xml_file: plate.xml
xml_file: fork.xml
xml_file: phillips_screwdriver.xml
xml_file: kinbody_55.xml
xml_file: hammer.xml
xml_file: kinbody_13.xml
xml_file: nontextured_12.xml
xml_file: nontextured_18.xml
xml_file: lego_duplo.xml
xml_file: spatula.xml
xml_file: kinbody_34.xml
xml_file: orange.xml
xml_file: lego_duplo_3.xml
xml_file: master_chef_can.xml
xml_file: foam_brick.xml
xml_file: tuna_fish_can.xml
xml_file: cups_3.xml
xml_file: tomato_soup_can.xml
xml_file: nontextured.xml
xml_file: kinbody_63.xml
xml_file: marbles.xml
xml_file: kinbody_57.xml
xml_file: kinbody_1.xml
xml_file: bleach_cleanser.xml
xml_file: cups_5.xml
xml_file: tennis_ball.xml
xml_file: kinbody_19.xml
xml_file: kinbody_28.xml
xml_file: kinbody_7.xml
xml_file: kinbody_38.xml
xml_file: non

### Add `free joint` tag for eash object `xml` files

In [3]:
import os
import xml.etree.ElementTree as ET

def add_free_joint(xml_filepath):
    # XML 파일을 파싱합니다.
    tree = ET.parse(xml_filepath)
    root = tree.getroot()

    # 모든 'body' 태그를 찾아서 'joint' 태그를 추가합니다.
    for body in root.iter('body'):
        # 새로운 'joint' 태그를 생성합니다.
        joint = ET.Element('joint')
        joint.set('name', f'{body.get("name")}:joint')
        joint.set('type', 'free')

        # 'joint' 태그를 'body' 태그 바로 아래에 삽입합니다.
        body.insert(0, joint)

    # 변경된 XML을 덮어씁니다.
    tree.write(xml_filepath)


dst_folder = 'asset/ycb_merged_v2/xml'  # xml 파일들이 있는 폴더

# 모든 xml 파일을 순회하면서 수정합니다.
for xml_file in os.listdir(dst_folder):
    if xml_file.endswith('.xml'):
        add_free_joint(os.path.join(dst_folder, xml_file))


### Appendix: Change the `material names` correspond to file name. (Not overlapped)

In [4]:
import os
import xml.etree.ElementTree as ET

def modify_xml(xml_filepath):
    # XML 파일 파싱
    tree = ET.parse(xml_filepath)
    root = tree.getroot()

    # 새로운 <compiler> 태그 생성
    compiler_tag = ET.Element('compiler')
    compiler_tag.set('meshdir', f'./{os.path.dirname(xml_filepath)}')
    compiler_tag.set('texturedir', f'./{os.path.dirname(xml_filepath)}')

    # 새로운 태그를 루트 아래에 삽입
    root.insert(1, compiler_tag)

    # 'default' 태그를 찾아서 제거
    for default_tag in root.findall('default'):
        root.remove(default_tag)

    # 'asset' 태그 내의 'material' 태그의 'name' 속성을 'texture' 속성과 동일하게 설정
    for asset_tag in root.findall('asset'):
        for material_tag in asset_tag.findall('material'):
            if 'texture' in material_tag.attrib:
                material_tag.set('name', material_tag.get('texture'))

    # 'worldbody' 태그 내부의 'body' 태그의 'name' 속성을 찾아 'geom' 태그의 'material' 속성을 그 값으로 설정
    for worldbody_tag in root.findall('worldbody'):
        for body_tag in worldbody_tag.findall('body'):
            body_name = body_tag.get('name')
            for geom_tag in body_tag.findall('geom'):
                geom_tag.set('material', body_name)

    # 수정된 트리를 다시 파일로 쓰기
    tree.write(xml_filepath)

merged_folder = 'asset/ycb_merged'  # 이를 ycb_merged 폴더로 변경해주세요.

# 모든 object 폴더를 순회
for object_folder in os.listdir(merged_folder):
    object_folder_path = os.path.join(merged_folder, object_folder)
    if os.path.isdir(object_folder_path):
        # 해당 object 폴더 내의 모든 XML 파일에 대해
        for xml_file in os.listdir(object_folder_path):
            if xml_file.endswith('.xml'):
                modify_xml(os.path.join(object_folder_path, xml_file))


### Appendix: not use priority of `folder names`

In [1]:
import os
import shutil

# 상위 디렉토리 경로 설정
src_folder = 'ycb_rename'
dest_folder = 'ycb_merged'

# 목적지 폴더 생성 (이미 존재하는 경우, 그냥 지나감)
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

# 상위 디렉토리 내의 모든 폴더에 대해
for root, dirs, files in os.walk(src_folder):
    for file in files:
        # 확장자가 obj, png, mtl인 파일만
        if file.endswith(('.obj', '.png', '.mtl')):
            # 원본 파일의 절대 경로
            src_file_path = os.path.join(root, file)
            # 목적지 파일의 절대 경로
            dest_file_path = os.path.join(dest_folder, file)
            
            # 동일한 이름의 파일이 목적지 폴더에 이미 존재하는 경우 이름을 변경
            if os.path.exists(dest_file_path):
                base, extension = os.path.splitext(file)
                i = 1
                while os.path.exists(dest_file_path):
                    file = f"{base}_{i}{extension}"
                    dest_file_path = os.path.join(dest_folder, file)
                    i += 1
            
            # 파일 복사
            shutil.copy2(src_file_path, dest_file_path)

print("All files are merged into the folder: ", dest_folder)



All files are merged into the folder:  ycb_merged
